In [0]:
!pip install google-generativeai python-binance --quiet
import google.generativeai as genai
from binance.client import Client
import os, json, datetime
from google.colab import drive
drive.mount('/content/drive')
BASE_PATH = '/content/drive/MyDrive/Нафаня бот'
os.makedirs(BASE_PATH, exist_ok=True)
gemini_key_path = os.path.join(BASE_PATH, 'Gemini', 'gemini_api_key.txt')
binance_config_path = os.path.join(BASE_PATH, 'Binance', 'binance_config.json')
config_path = os.path.join(BASE_PATH, 'nafani_config.json')
log_path = os.path.join(BASE_PATH, 'nafani_log.txt')
os.makedirs(os.path.join(BASE_PATH, 'Gemini'), exist_ok=True)
os.makedirs(os.path.join(BASE_PATH, 'Binance'), exist_ok=True)
if not os.path.exists(gemini_key_path):
    with open(gemini_key_path, 'w') as f:
        f.write('Вставь сюда свой ключ Gemini')
if not os.path.exists(binance_config_path):
    with open(binance_config_path, 'w') as f:
        json.dump({"api_key": "", "api_secret": ""}, f)
if not os.path.exists(config_path):
    with open(config_path, 'w') as f:
        json.dump({"auto_start": True, "log_level": "basic"}, f)
def log(message):
    with open(log_path, 'a') as f:
        f.write(f"[{datetime.datetime.now()}] {message}\n")
with open(config_path, 'r') as f:
    settings = json.load(f)
gemini_api_key = ''
if os.path.exists(gemini_key_path):
    with open(gemini_key_path, 'r') as f:
        gemini_api_key = f.read().strip()
if gemini_api_key and not gemini_api_key.startswith('Вставь'):
    try:
        genai.configure(api_key=gemini_api_key)
        model = genai.GenerativeModel('gemini-pro')
        gemini_ready = True
    except Exception as e:
        gemini_ready = False
        log(f'Ошибка подключения Gemini: {e}')
else:
    gemini_ready = False
binance_ready = False
binance_client = None
if os.path.exists(binance_config_path):
    with open(binance_config_path, 'r') as f:
        config = json.load(f)
    if config.get('api_key') and config.get('api_secret'):
        try:
            binance_client = Client(config['api_key'], config['api_secret'], testnet=True)
            binance_ready = True
        except Exception as e:
            binance_ready = False
            log(f'Ошибка подключения Binance: {e}')
print('=== ЯДРО НАФАНЯ ЗАПУЩЕНО ===')
print(f'Gemini готов: {'Да' if gemini_ready else 'Нет'}')
print(f'Binance готов: {'Да' if binance_ready else 'Нет'}')
print('Конфиг ядра загружен:', settings)
log('Ядро Нафаня запущено автоматически')
print('
Доступные команды:')
print('/статус - показать состояние систем')
print('/gemini - задать вопрос Gemini')
print('/binance - показать конфиг Binance')
print('/binance-setup - ввести ключи Binance')
print('/выход - завершить работу')
while True:
    cmd = input('
Введи команду: ')
    if cmd == '/выход':
        print('Нафаня завершает работу консоли.')
        log('Выход из консоли')
        break
    elif cmd == '/статус':
        status_msg = f'Gemini готов: {'Да' if gemini_ready else 'Нет'} | Binance готов: {'Да' if binance_ready else 'Нет'}'
        print(status_msg)
        log(f'Статус проверен: {status_msg}')
    elif cmd == '/gemini':
        if not gemini_ready:
            print('❌ Ключ Gemini не активирован. Вставь его в gemini_api_key.txt')
            log('Попытка использовать Gemini без ключа')
        else:
            user_msg = input('Введи вопрос для Gemini: ')
            try:
                response = model.generate_content(user_msg)
                print('Ответ Gemini:', response.text)
                log(f'Gemini запрос: {user_msg} | Ответ: {response.text}')
            except Exception as e:
                print('Ошибка при запросе:', e)
                log(f'Ошибка Gemini: {e}')
    elif cmd == '/binance':
        if os.path.exists(binance_config_path):
            with open(binance_config_path, 'r') as f:
                config = json.load(f)
            print('Binance конфиг:', config)
            log('Просмотр Binance конфига')
        else:
            print('Конфиг Binance не найден.')
            log('Ошибка: конфиг Binance отсутствует')
    elif cmd == '/binance-setup':
        api_key = input('Введи API Key Binance: ')
        api_secret = input('Введи API Secret Binance: ')
        with open(binance_config_path, 'w') as f:
            json.dump({'api_key': api_key, 'api_secret': api_secret}, f)
        print('Ключи Binance сохранены. Перезапусти ядро для обновления статуса.')
        log('Binance ключи обновлены вручную')
    else:
        print('Неизвестная команда. Попробуй снова.')
